# PETs/TETs – Hyperledger Aries / PySyft – Manufacturer 3 (Holder) 🛵

In [1]:
%%javascript
document.title = '🛵 Manufacturer3'

<IPython.core.display.Javascript object>

## PART 3: Connect with City to Analyze Data

**What:** Share encrypted data with City agent in a trust- and privacy-preserving manner

**Why:** Share data with City agent (e.g., to obtain funds)

**How:** <br>
1. [Initiate Manufacturer's AgentCommunicationManager (ACM)](#1)
2. [Connect anonymously with the City agent via a multi-use SSI invitation](#2)
3. [Prove Manufacturer3 Agent is a certified manufacturer via VCs](#3)
4. [Establish anonymous Duet Connection to share encrypted data](#4)

**Accompanying Agents and Notebooks:**
* City 🏙️️: `03_connect_with_Manufacturer.ipynb`
* Optional – Manufacturer1 🚗: `03_connect_with_city.ipynb`
* Optional – Manufacturer2 🚛: `03_connect_with_city.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [2]:
import os

import numpy as np
import pandas as pd
import syft as sy
import torch
from aries_cloudcontroller import AriesAgentController

from libs.agent_connection_manager import CredentialHolder

#### 0.2 – Variables

In [3]:
# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

---

<a id=1></a>

### 1 – Initiate Manufacturer3 Agent
#### 1.1 – Init ACA-PY agent controller

In [4]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

Initialising a controller with admin api at http://manufacturer3-agent:3021 and an api key of adminApiKey


#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [5]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.3 – Init ACM Credential Holder

In [6]:
# The CredentialHolder registers relevant webhook servers and event listeners
manufacturer3_agent = CredentialHolder(agent_controller)

# Verify if Manufacturer already has a VC
# (if there are manufacturer credentials, there is no need to execute the notebook)
manufacturer3_agent.get_credentials()

Successfully initiated AgentConnectionManager for a(n) Holder ACA-PY agent


{'results': [{'referent': 'isManufacturer-VC-M3',
   'attrs': {'manufacturerCountry': 'DE',
    'isManufacturer': 'TRUE',
    'manufacturerCity': 'City3',
    'manufacturerName': 'scooterManufacturer'},
   'schema_id': 'AkvQpXzutUhSeeiuZbVcbq:2:certify-manufacturer:0.0.1',
   'cred_def_id': 'AkvQpXzutUhSeeiuZbVcbq:3:CL:107773:default',
   'rev_reg_id': None,
   'cred_rev_id': None}]}

---

<a id=2></a>

### 2 – Establish a connection with the City agent 🏙️
A connection with the credential issuer (i.e., the authority agent) must be established before a VC can be received. In this scenario, the Manufacturer3 requests a connection with the Authority to be certified as an official city agency. Thus, the Manufacturer3 agent sends an invitation to the Authority. In real life, the invitation can be shared via video call, phone call, or E-Mail. In this PoC, this is represented by copy and pasting the invitation into the manufacturers' notebooks.

#### 2.1 Join invitation of City agent 🏙️
Copy and paste the multi-use invitation of the city agent, and establish a connection with them.

In [7]:
# Variables
alias = "undisclosedM3"
auto_accept = True

# Receive connection invitation
connection_id = manufacturer3_agent.receive_connection_invitation(alias=alias, auto_accept=auto_accept)

Please enter invitation received by external agent:


Invitation:  {     '@id': '9982b7e5-9fbc-4e6e-9828-48ae0bae50bd',     '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',     'label': 'City-Agency',     'recipientKeys': ['62C56o5uHkivZXqNuxTFH2WMLDBZ19P7tPbhFhWaTThp'],     'serviceEndpoint': 'https://f0ee-91-182-140-7.ngrok.io', }



---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  20a5be15-40be-41c0-8379-00c5ac7bb66a
State :  invitation (invitation-received)
Routing State : none
Connection with :  City-Agency
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  20a5be15-40be-41c0-8379-00c5ac7bb66a
State :  request (request-sent)
Routing State : none
Connection with :  City-Agency
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  20a5be15-40be-41c0-8379-00c5ac7bb66a
State :  response (response-received)
Routing State : none
Connection with :  City-Agency
Their Ro

<div style="font-size: 25px"><center><b>Break Point 2 / 3 / 4</b></center></div>
<div style="font-size: 50px"><center>🛵 ➡️ 🚗 / 🚛 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers. <br> If you have established a connection between the City and all Manufacturers, proceed to the City Notebook's Step 2.2</b></center>

---

<a id=3></a>
### 3 – Create Presentation to Send Proof Presentation

#### 3.1 – Create presentation that satisfies requirements of proof request
Before you can present a presentation, you must identify the presentation record which you wish to respond to with a presentation. To do so, the `prepare_presentation()` function runs through the following steps: 
1. Get all proof requests that were sent through `connection_id`
2. Get the most recent `presentation_exchange_id` and the corresponding `proof_request` from (1)
3. Get the restrictions the City agent defined in `proof_request` from (2)
4. Compare all VCs the Manufacturer3 agent has stored, and find (if available) a VC that satisfies the restrictions from (3)
5. Return a presentation dictionary from a VC from (4) that satisfies all requirements. Generally, a presentation consists of three classes of attributes: <br>
a. `requested_attributes`: Attributes that were signed by an issuer and have been revealed in the presentation process <br>
b. `self_attested_attributes`: Attributes that the prover has self attested to in the presentation object. <br>
c. `requested_predicates` (predicate proofs): Attribute values that have been proven to meet some statement. (TODO: Show how you can parse this information)

In [8]:
presentation, presentation_exchange_id = manufacturer3_agent.prepare_presentation(connection_id)

> Found proof_request with presentation_exchange_id 454ccdf5-9d2f-4188-bc01-a97a78a73e79
> Restrictions for a suitable proof: {'isManufacturer': {'requirements': {'schema_id': 'AkvQpXzutUhSeeiuZbVcbq:2:certify-manufacturer:0.0.1'}, 'request_attr_name': '0_isManufacturer_uuid'}}
> Attribute request for 'isManufacturer' can be satisfied by Credential with VC 'isManufacturer-VC-M3'
> Generate the proof presentation : 
{
    'requested_attributes': {
        '0_isManufacturer_uuid': {
            'cred_id': 'isManufacturer-VC-M3',
            'revealed': True,
        },
    },
    'requested_predicates': {},
    'self_attested_attributes': {},
}


#### 3.2 – Send Presentation

Send the presentation to the recipient of `presentation_exchange_id`

In [9]:
manufacturer3_agent.send_proof_presentation(presentation_exchange_id, presentation)


---------------------------------------------------------------------
Connection Webhook Event Received: Present-Proof Handler
Connection ID :  20a5be15-40be-41c0-8379-00c5ac7bb66a
Presentation Exchange ID :  454ccdf5-9d2f-4188-bc01-a97a78a73e79
Protocol State :  presentation_sent
Agent Role :  prover
Initiator :  external
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Present-Proof Handler
Connection ID :  20a5be15-40be-41c0-8379-00c5ac7bb66a
Presentation Exchange ID :  454ccdf5-9d2f-4188-bc01-a97a78a73e79
Protocol State :  presentation_acked
Agent Role :  prover
Initiator :  external
---------------------------------------------------------------------

Presentation Exchange ID: 454ccdf5-9d2f-4188-bc01-a97a78a73e79 is acknowledged by Relying Party


<div style="font-size: 25px"><center><b>Break Point 6 / 7 / 8</b></center></div>
<div style="font-size: 50px"><center>🛵 ➡️ 🚗 / 🚛 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers and run all cells between Steps 3 and 4.1 <br> If you have sent proof presentations from all manufacturers, proceed to the City Notebook's Step 3.3 </b></center>

---

<a id=4></a>
### 4 – Do Data Science
Assuming that the City agent will acknowledge the proofs and deem them to be correct, proceed by inviting the City agent to a Duet Connection.


#### 4.1 – Establish a Duet Connection with City Agent: Send Duet invitation
Duet is a package that allows you to exchange encrypted data and run privacy-preserving arithmetic operations on them (e.g., through homomorphic encryption or secure multiparty computation).

In [10]:
# Set up connection_id to use for duet connection
manufacturer3_agent._update_connection(connection_id=connection_id, is_duet_connection=True, reset_duet=True)

# Create duet invitation for city agent
duet = sy.launch_duet(credential_exchanger=manufacturer3_agent)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Sending Duet Token dff945582ab70d6a706a7ecd6a67d7f8
♫♫♫ > to Duet Partner City-Agency
♫♫♫ > via Connection ID 20a5be15-40be-41c0-8379-00c5ac7bb66a
♫♫♫ > Done!

♫♫♫ > STEP 2: Awaiting Duet Token from Duet Partner...

♫♫♫ > DONE! Partner's Duet Token: 5d019015000509faf31815e85ec1db50
♫♫♫ > Connecting...

♫♫♫ > CONNECTED!



[2021-11-27T15:56:06.083870+0000][CRITICAL][logger]][41] 'sympc'


ERROR:asyncio:Exception in callback AsyncIOEventEmitter._emit_run.<locals>._callback(<Task finishe...rror('sympc')>) at /opt/conda/lib/python3.9/site-packages/pyee/_asyncio.py:57
handle: <Handle AsyncIOEventEmitter._emit_run.<locals>._callback(<Task finishe...rror('sympc')>) at /opt/conda/lib/python3.9/site-packages/pyee/_asyncio.py:57>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/opt/conda/lib/python3.9/site-packages/pyee/_asyncio.py", line 64, in _callback
    self.emit("error", exc)
  File "/opt/conda/lib/python3.9/site-packages/pyee/_base.py", line 118, in emit
    self._emit_handle_potential_error(event, args[0] if args else None)
  File "/opt/conda/lib/python3.9/site-packages/pyee/_base.py", line 88, in _emit_handle_potential_error
    raise error
  File "/opt/conda/lib/python3.9/asyncio/tasks.py", line 256, in __step
    result = coro.send(None)
  File "/opt/co

ERROR:asyncio:Exception in callback AsyncIOEventEmitter._emit_run.<locals>._callback(<Task finishe...in the AST.')>) at /opt/conda/lib/python3.9/site-packages/pyee/_asyncio.py:57
handle: <Handle AsyncIOEventEmitter._emit_run.<locals>._callback(<Task finishe...in the AST.')>) at /opt/conda/lib/python3.9/site-packages/pyee/_asyncio.py:57>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/opt/conda/lib/python3.9/site-packages/pyee/_asyncio.py", line 64, in _callback
    self.emit("error", exc)
  File "/opt/conda/lib/python3.9/site-packages/pyee/_base.py", line 118, in emit
    self._emit_handle_potential_error(event, args[0] if args else None)
  File "/opt/conda/lib/python3.9/site-packages/pyee/_base.py", line 88, in _emit_handle_potential_error
    raise error
  File "/opt/conda/lib/python3.9/asyncio/tasks.py", line 256, in __step
    result = coro.send(None)
  File "/opt/co

(194, 24)T LIVE STATUS  -  Objects: 0  Requests: 0   Messages: 7  Request Handlers: 0                                


#### 4.2 - Load data to duet store

In [11]:
# Verify data store of duet
duet.store.pandas  # There should only be an MPC session statement by the City agent

""


Process data before loading it to the duet store. We take a synthetically created dataset of CO2 emission per trip across the City Agent's City (in this case Berlin, Germany).

In [12]:
# Get zipcode data (zipcode data from https://daten.odis-berlin.de/de/dataset/plz/)
df_zipcode = pd.read_csv("data/geo/berlin_zipcodes.csv").rename(columns={"plz":"zipcode"})
valid_zipcodes = list(df_zipcode.zipcode)
df_zipcode.head()

,zipcode
0,10115
1,10117
2,10119
3,10178
4,10179


In [13]:
# Get trip data
df_co2 = pd.read_csv("data/trips/data.csv", index_col=0)
df_co2 = df_co2[df_co2.zipcode.isin(valid_zipcodes)]
df_co2["hour"] = df_co2.timestamp.apply(lambda x: int(x[11:13]))
df_co2.head()

,i,vehicle_id,manufacturer_id,zipcode,timestamp,latlon,dist,seconds,co2_grams,total_dist,total_seconds,total_co2_grams,timestamp_tripstart,avg_kmperhour,avg_co2perkm,trip_id,hour
0,0,V25447006,V07918,12057,2021-08-19 19:31:30,"(52.4646074, 13.4467276)",0.00,0.0,0.00,0.00,0.0,0.0,2021-08-19 19:31:30,0.000000,0.000000,5,19
1,1,V25447006,V07918,12057,2021-08-19 19:31:31,"(52.464633, 13.446886)",0.01,1.0,1.20,0.01,1.0,1.0,2021-08-19 19:31:30,36.000000,100.000000,5,19
2,2,V25447006,V07918,12057,2021-08-19 19:31:36,"(52.4647272, 13.4475244)",0.04,5.0,5.68,0.05,6.0,7.0,2021-08-19 19:31:30,30.000000,140.000000,5,19
3,3,V25447006,V07918,12057,2021-08-19 19:31:39,"(52.4647421, 13.4477674)",0.02,3.0,2.66,0.07,9.0,10.0,2021-08-19 19:31:30,28.000000,142.857143,5,19
4,4,V25447006,V07918,12057,2021-08-19 19:31:41,"(52.4647652, 13.4479088)",0.01,2.0,1.25,0.08,11.0,11.0,2021-08-19 19:31:30,26.181818,137.500000,5,19


The trip data is then grouped by zipcode to sum the CO2 emission per hour per zipcode.


In [14]:
# Get hourly co2
df_hourly_co2 = df_co2[["zipcode", "hour","co2_grams"]].groupby(["zipcode", "hour"]).sum().reset_index()
df_hourly_co2 = df_hourly_co2.pivot(index=["zipcode"], columns=["hour"])["co2_grams"].replace(np.nan, 0)

# Get matrix that of shape (4085,25)
df_hourly_zipcode = df_zipcode.set_index("zipcode").reindex(columns=list(range(0,24))).replace(np.nan,0)#.reset_index()

# Merge dataframes together
df = df_hourly_zipcode.add(df_hourly_co2, fill_value=0)
print(df.shape)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
zipcode,,,,,,,,,,,,,,,,,,,,,
10115,0.00,0.0,0.00,0.0,0.0,48.87,0.0,50.89,0.0,0.0,...,0.0,83.91,40.95,0.00,35.21,0.0,0.0,0.00,0.00,32.42
10117,0.00,0.0,9.63,0.0,0.0,0.00,29.9,289.36,0.0,0.0,...,0.0,102.42,76.00,1.98,0.00,0.0,0.0,0.00,46.94,0.00
10119,73.41,0.0,0.00,0.0,0.0,154.80,0.0,0.00,0.0,0.0,...,0.0,114.03,0.00,0.00,166.20,0.0,0.0,61.65,0.00,165.29
10178,2.43,0.0,0.00,0.0,0.0,29.28,0.0,0.00,0.0,0.0,...,0.0,0.00,0.00,0.00,91.39,0.0,0.0,0.00,0.00,119.50
10179,36.46,0.0,0.00,0.0,0.0,0.00,0.0,46.77,0.0,0.0,...,0.0,0.00,74.36,16.41,0.00,0.0,0.0,0.00,0.00,145.70


Then, convert the dataset to a tensor, and upload the tensor with shape (194 x 24) to the duet data store

In [15]:
# Configure tensor
hourly_co2_torch = torch.tensor(df.values)
hourly_co2_torch = hourly_co2_torch.tag("hourly-co2-per-zip_2021-08-19")
hourly_co2_torch = hourly_co2_torch.describe("Total CO2 per Zipcode per Hour on August 19, 2021. Shape: zipcode (10115-14199) x hour (0-23) = 4085 x 24")

# Load tensor to datastore
hourly_co2_torch_pointer = hourly_co2_torch.send(duet, pointable=True)

# Verify datastore
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: f8a984c358ce42a29528d0ca335b0ba4>,[hourly-co2-per-zip_2021-08-19],"Total CO2 per Zipcode per Hour on August 19, 2...",<class 'torch.Tensor'>


#### 4.3 – Authorize City agent to `.reconstruct()` the data
Authorize the city agent to reconstruct the data once it is shared and joined with other manufacutrers' data

In [16]:
duet.requests.add_handler(
    #name="reconstruct",
    action="accept"
)

[2021-11-27T15:57:16.562123+0000][CRITICAL][logger]][41] Path sympc.session.Session not present in the AST.
[2021-11-27T15:57:16.563487+0000][CRITICAL][logger]][41] Path sympc.session.Session not present in the AST.
ERROR:asyncio:Exception in callback AsyncIOEventEmitter._emit_run.<locals>._callback(<Task finishe...in the AST.')>) at /opt/conda/lib/python3.9/site-packages/pyee/_asyncio.py:57
handle: <Handle AsyncIOEventEmitter._emit_run.<locals>._callback(<Task finishe...in the AST.')>) at /opt/conda/lib/python3.9/site-packages/pyee/_asyncio.py:57>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/opt/conda/lib/python3.9/site-packages/pyee/_asyncio.py", line 64, in _callback
    self.emit("error", exc)
  File "/opt/conda/lib/python3.9/site-packages/pyee/_base.py", line 118, in emit
    self._emit_handle_potential_error(event, args[0] if args else None)
  File "/opt/conda/l

---

### 5 – Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.
(Note: the terminating the controller will not terminate the Duet session).

In [ ]:
await agent_controller.terminate()

---

<div style="font-size: 25px"><center><b>Break Point 10 / 11 / 12</b></center></div>
<div style="font-size: 50px"><center>🛵 ➡️ 🚗 / 🚛 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers and run all cells between Steps 4.2 and 4.3 <br> If you have uploaded all data to the manufacturers' datastored, proceed to the City agent Step 4.2</b></center>

## 🔥🔥🔥 You can close this notebook now 🔥🔥🔥